In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 为出口到边缘的自动机器学习模型训练图像对象检测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl_image_object_detection_export_edge.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl_image_object_detection_export_edge.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl//automl_image_object_detection_export_edge.ipynb">
       <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI 工作台中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex AI SDK创建图像对象检测模型，并将其导出为Edge模型，使用AutoML模型。

### 目标

在本教程中，您将使用Vertex SDK从Python脚本创建一个AutoML图像对象检测模型，然后将该模型导出为TFLite格式的Edge模型。您还可以使用`gcloud`命令行工具或在云控制台上在线创建AutoML模型。

本教程使用以下Google Cloud ML服务：

- Vertex AI `Datasets`
- AutoML Image

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 从`Model`资源中将`Edge`模型导出到Cloud Storage。
- 在本地下载模型。
- 进行本地预测。

### 数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[OpenImages数据集](https://www.tensorflow.org/datasets/catalog/open_images_v4)中的沙拉类别。这个数据集不需要任何特征工程。在本教程中使用的数据集版本存储在一个公共云存储桶中。训练模型可以预测图像中五种项目类别中沙拉项目的边界框位置和对应类型：沙拉、海鲜、番茄、烘焙食品、或奶酪。

### 费用

本教程使用谷歌云的可计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI的定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage的定价](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/) 基于您的预期使用量生成费用估算。

安装最新版本的Python的Vertex AI SDK。

In [ ]:
import os

! pip3 install --upgrade --quiet google-cloud-aiplatform

if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

### 仅限Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在你开始之前

### 设置您的项目 ID

**如果您不知道您的项目 ID**，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

### 区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动验证。请按照以下相关说明进行操作。

1. 顶点 AI 工作台
* 什么也不需要做，因为你已经通过验证。

2. 本地 JupyterLab 实例，取消注释并运行：

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在 https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上向您的服务帐号授予云存储权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库并定义常量

In [ ]:
import os

import google.cloud.aiplatform as aiplatform

## 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化用于 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

# 教程

现在你已经准备好开始创建你自己的AutoML图像目标检测模型了。

云存储培训数据的位置。

现在将变量`IMPORT_FILE`设置为Cloud Storage中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/vision/salads.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的沙拉数据集的版本，使用CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`ImageDataset`类的`create`方法为`Dataset`资源创建数据集，需要提供以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：要将数据项导入`Dataset`资源的一个或多个数据集索引文件列表。
- `import_schema_uri`：数据项的数据标记模式。

这个操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.ImageDataset.create(
    display_name="Salads",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.bounding_box,
)

print(dataset.resource_name)

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1) 创建一个训练管道，2) 运行该管道。

#### 创建训练管道

使用 `AutoMLImageTrainingJob` 类创建一个AutoML训练管道，具有以下参数：

- `display_name`：`TrainingJob` 资源的人类可读名称。
- `prediction_type`：训练模型的任务类型。
  - `classification`：图像分类模型。
  - `object_detection`：图像目标检测模型。
- `multi_label`：如果是分类任务，是单标签 (`False`) 还是多标签 (`True`)。
- `model_type`：部署模型的类型。
  - `CLOUD`：部署在Google Cloud上。
  - `CLOUD_HIGH_ACCURACY_1`：针对精度优化而非延迟部署在Google Cloud上。
  - `CLOUD_LOW_LATENCY_`：针对延迟而非精度优化部署在Google Cloud上。
  - `MOBILE_TF_VERSATILE_1`：部署在边缘设备上。
  - `MOBILE_TF_HIGH_ACCURACY_1`：针对精度优化而非延迟部署在边缘设备上。
  - `MOBILE_TF_LOW_LATENCY_1`：针对延迟而非精度优化部署在边缘设备上。
- `base_model`：（可选）从现有的 `Model` 资源进行迁移学习 --仅支持图像分类。

实例化的对象是训练作业的DAG（有向无环图）。

In [ ]:
dag = aiplatform.AutoMLImageTrainingJob(
    display_name="salads",
    prediction_type="object_detection",
    multi_label=False,
    model_type="MOBILE_TF_LOW_LATENCY_1",
    base_model=None,
)

print(dag)

#### 运行训练流水线

接下来，您可以通过调用`run`方法来运行DAG从而开始训练作业，具体参数如下：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 1小时）。
- `disable_early_stopping`：如果`True`，服务可能认为无法进一步提高模型目标测量，因此训练可能在使用完整预算之前完成。

`run`方法完成后将返回`Model`资源。

训练流水线的执行将最多需要60分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="salads",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=20000,
    disable_early_stopping=False,
)

## 回顾模型评估分数

在模型训练完成后，您可以使用 `list_model_evaluations()` 方法查看其评估分数。该方法将返回每个评估切片的迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

将模型导出为Edge模型

您可以将AutoML图像目标检测模型导出为`Edge`模型，然后将其定制部署到边缘设备或在本地下载。使用`export_model()`方法将模型导出到云存储，该方法接受以下参数：

- `artifact_destination`：存储SavedFormat模型工件的云存储位置。
- `export_format_id`：要保存模型格式的格式。对于AutoML图像目标检测，有以下选项：
   - `tf-saved-model`：用于部署到容器的TensorFlow SavedFormat。
   - `tflite`：用于部署到边缘或移动设备的TensorFlow Lite。
   - `edgetpu-tflite`：用于TPU的TensorFlow Lite。
   - `tf-js`：用于Web客户端的TensorFlow。
   - `coral-ml`：用于Coral设备。

- `sync`：是否同步或异步执行操作。

In [ ]:
response = model.export_model(
    artifact_destination=BUCKET_URI, export_format_id="tflite", sync=True
)

model_package = response["artifactOutputUri"]

#### 下载TFLite模型文件

现在您已经导出了您的模型的TFLite版本，您可以在本地测试导出的模型，但首先需要从云存储中下载它。

In [ ]:
! gsutil ls $model_package
# Download the model artifacts
! gsutil cp -r $model_package tflite

tflite_path = "tflite/model.tflite"

实例化一个TFLite解释器

模型的TFLite版本不是TensorFlow SavedModel格式。您不能直接使用像predict()这样的方法。相反，您需要使用TFLite解释器。您必须首先为TFLite模型设置解释器，步骤如下：

- 为TFLite模型实例化一个TFLite解释器。
- 指示解释器为模型分配输入和输出张量。
- 获取关于模型输入和输出张量的详细信息，这是预测所需的。

In [ ]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]["shape"]

print("input tensor shape", input_shape)

获取测试项目

您将从数据集中选择任意的示例作为测试项目。不必担心该示例很可能在训练模型时被使用过 — 我们只是想演示如何进行预测。

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n1
test_item = test_items[0].split(",")[0]

with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()
test_image = tf.io.decode_jpeg(content)
print("test image shape", test_image.shape)

test_image = tf.image.resize(test_image, (192, 192))
print("test image shape", test_image.shape, test_image.dtype)

test_image = tf.cast(test_image, dtype=tf.uint8).numpy()

#### 使用 TFLite 模型进行预测

最后，您将使用您的 TFLite 模型进行预测，步骤如下：

- 将测试图像转换为单个图像批次（`np.expand_dims`）。
- 将输入张量设置为解释器的单个图像批次（`data`）。
- 调用解释器。
- 检索预测的 softmax 概率（`get_tensor`）。
- 确定哪个标签具有最高的概率（`np.argmax`）。

In [ ]:
import numpy as np

data = np.expand_dims(test_image, axis=0)

interpreter.set_tensor(input_details[0]["index"], data)

interpreter.invoke()

softmax = interpreter.get_tensor(output_details[0]["index"])

label = np.argmax(softmax)

print(label)

清理工作

为了清理本项目中使用的所有谷歌云资源，您可以删除您用于本教程的[谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = False

# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML trainig job
dag.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI